In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pandas as pd
import pymc as pm
import numpy as np
import arviz as az
sys.path.append("../src/")
from models import create_blr_full_pooling, create_blr_no_pooling, create_blr_partial_pooling, create_blr_partial_pooling_gp
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

In [3]:
from sklearn.preprocessing import LabelEncoder

# Load preprocessed data
data = pd.read_csv("../data/california_housing_pre.csv")
data.dropna(inplace=True)
data = data.sample(100)
y = data.pop("median_house_value")

# Convert "county" column to integer
label_encoder = LabelEncoder()
data["county_nr"] = label_encoder.fit_transform(data["county"])

data = data.loc[:, ["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup", "county_lon", "county_lat", "county_nr"]]

data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 3400 to 6340
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   MedInc      100 non-null    float64
 1   HouseAge    100 non-null    float64
 2   AveRooms    100 non-null    float64
 3   AveBedrms   100 non-null    float64
 4   Population  100 non-null    float64
 5   AveOccup    100 non-null    float64
 6   county_lon  100 non-null    float64
 7   county_lat  100 non-null    float64
 8   county_nr   100 non-null    int32  
dtypes: float64(8), int32(1)
memory usage: 7.4 KB


In [4]:
# split into train and test data
x_train, x_test, y_train, y_test = train_test_split(data, y, random_state=0, test_size=0.3)
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,county_lon,county_lat,county_nr
3400,4.6944,30.0,6.030019,0.962477,1700.0,3.189493,-118.418613,34.251168,31
4553,1.1979,33.0,2.020725,1.031088,435.0,2.253886,-118.359326,33.993978,19
5726,4.9297,31.0,5.309255,1.038375,1895.0,2.138826,-118.418613,34.251168,31
11202,1.5903,16.0,3.690763,1.016064,912.0,3.662651,-117.859134,33.828326,1
14119,1.6034,30.0,3.851927,1.107505,1620.0,3.286004,-117.102163,32.823399,30


### Random Forest Baseline

In [5]:
rfr = RandomForestRegressor()
rfr.fit(x_train, y_train)
y_pred = rfr.predict(x_test)
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))

MAE: 0.5797302666666666
MSE: 0.5435027443227252


### BLR - Full Pooling

In [6]:
# create bayesian linear regression model
blr_full_pooling = create_blr_full_pooling(x_train, y_train)

In [7]:
with blr_full_pooling:
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, beta_medinc, beta_house_age, beta_ave_rooms, beta_ave_bedrms, beta_population, beta_ave_occup, sigma]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 69 seconds.


#### Predict on test set

In [8]:
with blr_full_pooling:
    new_data = {"medinc": x_test.MedInc, 
                "house_age": x_test.HouseAge, 
                "ave_rooms": x_test.AveRooms,
                "ave_bedrms": x_test.AveBedrms,
                "population": x_test.Population,
                "ave_occup": x_test.AveOccup,
                "median_house_value": np.zeros(shape=y_test.shape)}  
    pm.set_data(new_data)
    idata.extend(pm.sample_posterior_predictive(idata))

y_pred = idata.posterior_predictive.y.mean(axis=0).mean(axis=0).values
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))


Sampling: [y]


MAE: 0.48972751495240924
MSE: 0.377516043133102


### BLR - no pooling

In [9]:
coords = {"spatial_groups": list(data.county_nr.unique())}
blr_no_pooling = create_blr_no_pooling(x_train, y_train, coords, spatial_grouping_var="county_nr")

In [10]:
with blr_no_pooling:
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, beta_medinc, beta_house_age, beta_ave_rooms, beta_ave_bedrms, beta_population, beta_ave_occup, sigma]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 110 seconds.


In [11]:
with blr_no_pooling:
    new_data = {"medinc": x_test.MedInc, 
                "house_age": x_test.HouseAge, 
                "ave_rooms": x_test.AveRooms,
                "ave_bedrms": x_test.AveBedrms,
                "population": x_test.Population,
                "ave_occup": x_test.AveOccup,
                "spatial_group_idx": x_test.county_nr,
                "median_house_value": np.zeros(shape=y_test.shape)}  
    pm.set_data(new_data)
    idata.extend(pm.sample_posterior_predictive(idata))

y_pred = idata.posterior_predictive.y.mean(axis=0).mean(axis=0).values
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))

Sampling: [y]


MAE: 0.46378198354372
MSE: 0.336643044603478


### BLR - partial pooling

In [12]:
coords = {"spatial_groups": list(data.county_nr.unique())}
blr_partial_pooling = create_blr_partial_pooling(x_train, y_train, coords, spatial_grouping_var="county_nr")

In [13]:
with blr_partial_pooling:
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept_mu, intercept_sigma, intercept, beta_medinc, beta_house_age, beta_ave_rooms, beta_ave_bedrms, beta_population, beta_ave_occup, sigma]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 61 seconds.


In [14]:
with blr_partial_pooling:
    new_data = {"medinc": x_test.MedInc, 
                "house_age": x_test.HouseAge, 
                "ave_rooms": x_test.AveRooms,
                "ave_bedrms": x_test.AveBedrms,
                "population": x_test.Population,
                "ave_occup": x_test.AveOccup,
                "spatial_group_idx": x_test.county_nr,
                "median_house_value": np.zeros(shape=y_test.shape)}  
    pm.set_data(new_data)
    idata.extend(pm.sample_posterior_predictive(idata))

y_pred = idata.posterior_predictive.y.mean(axis=0).mean(axis=0).values
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))

Sampling: [y]


MAE: 0.45031176039961485
MSE: 0.31584203741414946


In [19]:
idata.posterior.intercept

<xarray.DataArray 'intercept' (chain: 4, draw: 1000, spatial_groups: 49)>
array([[[ 1.51505642,  1.38063946,  0.74814357, ...,  1.41533924,
          1.38448885,  0.91755314],
        [ 1.42830655,  1.04443473,  0.4038864 , ...,  1.61952774,
          0.93617962,  1.4310583 ],
        [ 1.13591659,  2.01291849,  0.95916963, ...,  0.8568057 ,
          0.97719337,  0.20385956],
        ...,
        [ 0.94256118,  1.03234808,  0.75331972, ...,  0.91583016,
          1.16206023,  0.85675307],
        [ 0.92405647,  0.92813509,  0.8685433 , ...,  1.17349789,
          0.79080393,  1.01858748],
        [ 0.94199228,  0.94579375,  1.01852975, ...,  0.81421809,
          0.91451666,  0.92163481]],

       [[ 1.0666645 ,  1.90344108,  0.66012758, ...,  0.72401052,
          2.02593323,  1.20325165],
        [ 1.49781828,  0.42741321,  0.78116317, ...,  2.00769602,
          0.99656158,  0.50115429],
        [ 1.19780934,  1.59118852,  0.12648266, ...,  1.53831898,
          0.56104921,  1.05638013],
...
        [ 0.3976775 ,  0.78125465,  0.21457599, ...,  0.31714924,
          0.52851553,  0.66397708],
        [ 0.80714281,  0.42609873,  0.38003272, ...,  1.03878275,
          0.44816681, -0.02275094],
        [ 0.39290136,  0.55500311,  0.53757075, ...,  0.26134084,
          0.57442893,  0.92395042]],

       [[ 0.96919092,  1.43841576,  1.2404992 , ...,  1.15354766,
          1.60448085,  1.30448303],
        [ 1.06728114,  1.44427733,  0.69398185, ...,  1.86947831,
          1.31645021,  0.76386748],
        [ 1.49615114,  1.30416992,  0.917138  , ...,  1.59073657,
          1.25283707,  0.72096452],
        ...,
        [ 1.03651844,  0.44219518,  0.72929071, ...,  0.90930006,
          0.42634108,  0.21391697],
        [ 0.98535999,  0.82879384,  0.26609843, ...,  0.99074332,
          0.99400837,  1.05677198],
        [ 0.56708255,  0.66191476,  0.30156972, ...,  1.39283445,
          1.8583404 ,  0.01907645]]])
Coordinates:
  * chain           (chain) int32 0 1 2 3
  * draw            (draw) int32 0 1 2 3 4 5 6 7 ... 993 994 995 996 997 998 999
  * spatial_groups  (spatial_groups) int32 31 19 1 30 48 40 ... 45 17 35 25 16

### BLR - partial pooling gaussian priors

In [15]:
unique_county = data.groupby('county_nr').agg({'county_lat': 'first', 'county_lon': 'first'}).reset_index()
coords = {"spatial_groups": list(unique_county.county_nr)}
county_coordinates = unique_county.loc[:, ["county_lon", "county_lat"]].values

In [16]:

blr_partial_pooling_gp = create_blr_partial_pooling_gp(x_train, y_train, coords, spatial_grouping_var="county_nr", spatial_groups_coordinates=county_coordinates)

In [17]:
with blr_partial_pooling_gp:
    idata_gp = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


ValueError: array must not contain infs or NaNs
Apply node that caused the error: Cholesky{lower=True, destructive=False, on_error='raise'}(Elemwise{Composite{((exp((i0 * clip((i1 + i2 + i3), i4, i5))) * i6) + (i7 * i8))}}[(0, 1)].0)
Toposort index: 63
Inputs types: [TensorType(float64, (?, ?))]
Inputs shapes: [(49, 49)]
Inputs strides: [(392, 8)]
Inputs values: ['not shown']
Outputs clients: [[CGemv{inplace}(AllocEmpty{dtype='float64'}.0, TensorConstant{1.0}, Cholesky{lower=True, destructive=False, on_error='raise'}.0, intercept_gp_rotated_, TensorConstant{0.0})]]

Backtrace when the node is created (use PyTensor flag traceback__limit=N to make it longer):
  File "c:\Users\David\anaconda3\envs\pymc_env\Lib\site-packages\IPython\core\async_helpers.py", line 129, in _pseudo_sync_runner
    coro.send(None)
  File "c:\Users\David\anaconda3\envs\pymc_env\Lib\site-packages\IPython\core\interactiveshell.py", line 3203, in run_cell_async
    has_raised = await self.run_ast_nodes(code_ast.body